In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils.data_utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [16]:
data = pd.read_csv('/content/dataset_spam.csv')
data.tail()

,content,label
6663,f a n s b e t i n g agent bola terpercaya di i...,spam
6664,pasti ada campur tangan ulah orang dalam karen...,normal
6665,mauuu puunyaa kullittt puutiiihh secaaraa inns...,spam
6666,bagi aku kaaa dikit aja,normal
6667,mila sipitt allah tuh benci sama orang yg meru...,normal


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6668 entries, 0 to 6667
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  6668 non-null   object
 1   label    6668 non-null   object
dtypes: object(2)
memory usage: 104.3+ KB


In [22]:
X = data.content
Y = data.label
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)
Y = Y.reshape(-1,1)

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [27]:
# conversi teks ke sequence
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

In [28]:
# define RNN
def RNN():
    inputs = Input(name='inputs', shape=[max_len])
    layer = Embedding(max_words, 50, input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs, outputs=layer)
    return model

In [30]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 150, 50)           50000     
                                                                 
 lstm_1 (LSTM)               (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation_2 (Activation)   (None, 256)               0         
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257 

In [31]:
model.fit(sequences_matrix, Y_train,batch_size=128, epochs=10,
          validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.0001)])

Epoch 1/10
30/30 [==============================] - 12s 281ms/step - loss: 0.3660 - accuracy: 0.8666 - val_loss: 0.2752 - val_accuracy: 0.9261
Epoch 2/10
30/30 [==============================] - 8s 254ms/step - loss: 0.1929 - accuracy: 0.9443 - val_loss: 0.2363 - val_accuracy: 0.9315
Epoch 3/10
30/30 [==============================] - 11s 380ms/step - loss: 0.1258 - accuracy: 0.9606 - val_loss: 0.1512 - val_accuracy: 0.9518
Epoch 4/10
30/30 [==============================] - 9s 299ms/step - loss: 0.0981 - accuracy: 0.9692 - val_loss: 0.1750 - val_accuracy: 0.9507


In [33]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences, maxlen=max_len)

In [35]:
accr = model.evaluate(test_sequences_matrix, Y_test)

63/63 [==============================] - 2s 29ms/step - loss: 0.1775 - accuracy: 0.9510


In [36]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.177
  Accuracy: 0.951
